# V7 Modeli

**Veri Hazırlama**

In [ ]:
import pandas as pd
import numpy as np
import gc
import os

print("Adım 1 (v7 - Modeli): Veri Hazırlama Başladı.")

IN_TRAIN_PATH = '/content/datathon/train.csv'
IN_TEST_PATH = '/content/datathon/test.csv'

OUT_DIR = '/content/datathon/processed/'
os.makedirs(OUT_DIR, exist_ok=True)

OUT_TRAIN_PATH = OUT_DIR + 'train_processed_v7.csv'
OUT_TEST_PATH = OUT_DIR + 'test_processed_v7.csv'

# --- Veri Yükleme ve Ön Bilgiler (v6 ile aynı) ---
try:
    df_train = pd.read_csv(IN_TRAIN_PATH, parse_dates=['event_time'])
    df_test = pd.read_csv(IN_TEST_PATH, parse_dates=['event_time'])
    print("Ham veri setleri başarıyla yüklendi.")

    # Analiz için ön bilgiler
    train_users = set(df_train['user_id'])
    test_users = set(df_test['user_id'])
    common_users = train_users.intersection(test_users)
    train_products = set(df_train['product_id'])
    new_products_in_test = set(df_test['product_id']) - train_products
    train_session_users = df_train.groupby('user_session')['user_id'].apply(set)
    test_session_users = df_test.groupby('user_session')['user_id'].apply(set)
    common_sessions = set(train_session_users.index).intersection(set(test_session_users.index))
    verified_leaked_sessions = {sid for sid in common_sessions if train_session_users[sid] == test_session_users[sid]}
    print("Analizler için ön bilgiler hesaplandı.")
except Exception as e:
    print(f"Hata: {e}.")
    exit()

# --- Anomali Temizleme, Kullanıcı Özellikleri (v6 ile aynı) ---
# ... (Bu kısımlar önceki script ile birebir aynı, o yüzden tekrar eklemiyorum)
def fix_anomalous_sessions(df, data_type='train'):
    session_user_counts = df.groupby('user_session')['user_id'].nunique()
    anomalous_sessions = session_user_counts[session_user_counts > 1].index
    if len(anomalous_sessions) > 0:
        anomalous_indices = df['user_session'].isin(anomalous_sessions)
        df['user_session_corrected'] = df['user_session']
        df.loc[anomalous_indices, 'user_session_corrected'] = df.loc[anomalous_indices, 'user_session'] + '_' + df.loc[anomalous_indices, 'user_id']
        df.drop('user_session', axis=1, inplace=True)
        df.rename(columns={'user_session_corrected': 'user_session'}, inplace=True)
    return df
df_train = fix_anomalous_sessions(df_train, 'train')
df_test = fix_anomalous_sessions(df_test, 'test')

df_combined = pd.concat([df_train.drop('session_value', axis=1), df_test], ignore_index=True)
user_features = df_combined.groupby('user_id').agg(
    user_total_events=('event_type', 'count'), user_unique_products_viewed=('product_id', 'nunique'),
    user_first_seen=('event_time', 'min'), user_last_seen=('event_time', 'max')
)
user_features['user_lifespan_days'] = (user_features['user_last_seen'] - user_features['user_first_seen']).dt.days
user_buy_counts = df_combined[df_combined['event_type'] == 'BUY'].groupby('user_id').size()
user_features['user_buy_count'] = user_buy_counts
user_features['user_buy_count'].fillna(0, inplace=True)
user_features['user_purchase_rate'] = user_features['user_buy_count'] / user_features['user_total_events']
user_features.drop(['user_first_seen', 'user_last_seen'], axis=1, inplace=True)
del df_combined
gc.collect()

# --- 4. ADIM: SEANS BAZLI ÖZELLİKLER (v7) ---
def create_session_features_v7(df, data_type='train'):
    print(f"\n{data_type} verisi için seans bazlı özellik mühendisliği (v7) başlıyor...")

    # --- YENİ EKLENEN KISIM 1: HESAPLAMA ---
    # Her seans içinde olayları zamana göre sırala ve sıra numarası ver
    df['event_order'] = df.groupby('user_session').cumcount() + 1
    session_event_counts = df['user_session'].map(df['user_session'].value_counts())
    df['event_order_pct'] = df['event_order'] / session_event_counts

    session_products = df.groupby('user_session')['product_id'].apply(set)
    event_type_counts = pd.crosstab(df['user_session'], df['event_type'])
    all_event_types = ['VIEW', 'ADD_CART', 'REMOVE_CART', 'BUY']
    for event in all_event_types:
        if event not in event_type_counts.columns:
            event_type_counts[event] = 0
    event_type_counts.columns = [f'{col.lower()}_count' for col in event_type_counts.columns]

    session_features = df.groupby('user_session').agg(
        user_id=('user_id', 'first'),
        event_count=('event_type', 'count'),
        unique_products=('product_id', 'nunique'),
        unique_categories=('category_id', 'nunique'),
        session_duration_seconds=('event_time', lambda x: (x.max() - x.min()).total_seconds()),
        avg_day_of_week=('event_time', lambda x: x.dt.dayofweek.mean()),
        avg_hour=('event_time', lambda x: x.dt.hour.mean()),
        # --- YENİ EKLENEN KISIM 2: ÖZETLEME ---
        avg_event_order=('event_order', 'mean'),
        avg_event_order_pct=('event_order_pct', 'mean')
    )

    # Analiz bazlı özellikler
    session_features['is_common_user'] = session_features['user_id'].isin(common_users).astype(int)
    session_features['is_leaked_session'] = session_features.index.isin(verified_leaked_sessions).astype(int)
    if data_type == 'test':
        session_features['has_new_product'] = [1 if not session_products[sid].isdisjoint(new_products_in_test) else 0 for sid in session_features.index]
    else:
        session_features['has_new_product'] = 0

    df_session = pd.concat([session_features, event_type_counts], axis=1)

    epsilon = 1e-6
    df_session['view_to_add_cart_rate'] = df_session['add_cart_count'] / (df_session['view_count'] + epsilon)
    df_session['add_cart_to_buy_rate'] = df_session['buy_count'] / (df_session['add_cart_count'] + epsilon)
    df_session['view_to_buy_rate'] = df_session['buy_count'] / (df_session['view_count'] + epsilon)
    df_session['net_cart_additions'] = df_session['add_cart_count'] - df_session['remove_cart_count']
    df_session['did_purchase'] = (df_session['buy_count'] > 0).astype(int)

    return df_session

df_session_train = create_session_features_v7(df_train, 'train')
df_session_test = create_session_features_v7(df_test, 'test')

# --- 5. ADIM ve sonrası (v6 ile aynı) ---
print("\nKullanıcı ve seans özellikleri birleştiriliyor...")
df_session_train = df_session_train.merge(user_features, on='user_id', how='left').set_index(df_session_train.index)
df_session_test = df_session_test.merge(user_features, on='user_id', how='left').set_index(df_session_test.index)
df_session_train.drop('user_id', axis=1, inplace=True)
df_session_test.drop('user_id', axis=1, inplace=True)

print("\nEn önemli etkileşim özellikleri oluşturuluyor...")
for df in [df_session_train, df_session_test]:
    df['buy_x_hour'] = df['buy_count'] * df['avg_hour']
    df['buy_x_unique_products'] = df['buy_count'] * df['unique_products']
    df['buy_x_user_purchase_rate'] = df['buy_count'] * df['user_purchase_rate']

session_value = df_train.groupby('user_session')['session_value'].first()
df_session_train['session_value'] = session_value

df_session_train.to_csv(OUT_TRAIN_PATH)
df_session_test.to_csv(OUT_TEST_PATH)

print("\nAdım 1 (v7) Tamamlandı: 'train_processed_v7.csv' ve 'test_processed_v7.csv' dosyaları oluşturuldu.")

Adım 1 (v7 - Nihai Sürüm): Veri Hazırlama Başladı.
Ham veri setleri başarıyla yüklendi.
Analizler için ön bilgiler hesaplandı.


/tmp/ipython-input-3029964225.py:61: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_features['user_buy_count'].fillna(0, inplace=True)



train verisi için seans bazlı özellik mühendisliği (v7) başlıyor...

test verisi için seans bazlı özellik mühendisliği (v7) başlıyor...

Kullanıcı ve seans özellikleri birleştiriliyor...

En önemli etkileşim özellikleri oluşturuluyor...

Adım 1 (v7) Tamamlandı: 'train_processed_v7.csv' ve 'test_processed_v7.csv' dosyaları oluşturuldu.


**Model Eğitme**

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

print("Adım 2: Model Eğitimi Başlandı.")

MODEL_DIR = "/content/models/V7"
os.makedirs(MODEL_DIR, exist_ok=True)

IN_TRAIN_PATH = "/content/datathon/processed/train_processed_v7.csv"

FEATURE_IMPORTANCE_DIR_OUT = MODEL_DIR + "/feature_importance/IN/"
os.makedirs(FEATURE_IMPORTANCE_DIR_OUT, exist_ok=True)

OUT_FEATURES_PATH = FEATURE_IMPORTANCE_DIR_OUT + "importance.json"

# --- İşlenmiş Veriyi Yükleme ---
try:
    df_train = pd.read_csv(IN_TRAIN_PATH, index_col='user_session')
    print("İşlenmiş train verisi ('train_processed.csv') yüklendi.")
except FileNotFoundError:
    print("Hata: 'train_processed.csv' bulunamadı. Lütfen önce '1_data_preparation_v7.py' scriptini çalıştırın.")
    exit()

# --- Modelleme için Veriyi Hazırlama ---
y = df_train['session_value']
X = df_train.drop(['session_value'], axis=1)


# Hedef değişkene log dönüşümü
y_log = np.log1p(y)

# --- Zaman Bazlı Doğrulama (Time-Based Validation) ---
X_train, X_val, y_train_log, y_val_log = train_test_split(
    X, y_log, test_size=0.2, shuffle=False # shuffle=False zaman serisi doğrulama için kritik!
)
print(f"Train seti boyutu: {X_train.shape[0]}, Validation seti boyutu: {X_val.shape[0]}")

# --- CatBoost Modelini Eğitme ve Değerlendirme ---
print("CatBoost Modeli eğitimi başlıyor...")

best_params = {
    'learning_rate': 0.06007502637954865,
    'depth': 4,
    'l2_leaf_reg': 2.5440079813104126,
    'colsample_bylevel': 0.8424763199702767,
    'min_child_samples': 29,
    'objective': 'RMSE',
    'random_seed': 42,
    'verbose': 500
}

cat_model = CatBoostRegressor(
    **best_params,
    iterations=4500,
    eval_metric='RMSE',
    early_stopping_rounds=300
)


cat_model.fit(
    X_train, y_train_log,
    eval_set=(X_val, y_val_log)
)

# --- Performans Değerlendirme ---
val_preds_log = cat_model.predict(X_val)
val_preds = np.expm1(val_preds_log)
val_preds[val_preds < 0] = 0
y_val = np.expm1(y_val_log)

validation_mse = mean_squared_error(y_val, val_preds)
print(f"\nValidation Seti Üzerindeki MSE Skoru (CatBoost): {validation_mse:.4f}")
print(f"Validation Seti Üzerindeki RMSE Skoru (CatBoost): {np.sqrt(validation_mse):.4f}")

# --- Final Modelini Eğitme ve Kaydetme ---
print("\nFinal CatBoost modeli tüm train verisi üzerinde eğitiliyor...")
final_model = CatBoostRegressor(
    **best_params,
    iterations=cat_model.get_best_iteration()
)
final_model.fit(X, y_log)

features = X.columns.tolist()

# Feature importance as a DataFrame and save in a format that works
fi = final_model.get_feature_importance(prettified=False)
df_fi = pd.DataFrame({"feature": features, "importance": fi})
df_fi = df_fi.sort_values("importance", ascending=False).reset_index(drop=True)

# Prefer parquet, fallback to CSV if parquet not available, always also save JSON

df_fi.to_json(OUT_FEATURES_PATH, orient="records")



out_dir = MODEL_DIR + "/"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "catboost_model_v7.cbm")

# Modeli kaydetme
final_model.save_model(out_path)

print("\nAdım 2 Tamamlandı: 'catboost_model_v7.cbm' dosyası olarak model kaydedildi.")

Adım 2: Model Eğitimi Başlandı.
İşlenmiş train verisi ('train_processed.csv') yüklendi.
Train seti boyutu: 56605, Validation seti boyutu: 14152
CatBoost Modeli eğitimi başlıyor...
0:	learn: 0.7430447	test: 0.7430925	best: 0.7430925 (0)	total: 65.1ms	remaining: 4m 52s
500:	learn: 0.4366534	test: 0.4342448	best: 0.4342448 (500)	total: 4.74s	remaining: 37.8s
1000:	learn: 0.4338461	test: 0.4331916	best: 0.4331916 (1000)	total: 11.7s	remaining: 41.1s
1500:	learn: 0.4321036	test: 0.4329846	best: 0.4329744 (1462)	total: 16.3s	remaining: 32.5s
2000:	learn: 0.4308468	test: 0.4330191	best: 0.4329602 (1779)	total: 23.2s	remaining: 28.9s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.4329602147
bestIteration = 1779

Shrink model to first 1780 iterations.

Validation Seti Üzerindeki MSE Skoru (CatBoost): 272.3057
Validation Seti Üzerindeki RMSE Skoru (CatBoost): 16.5017

Final CatBoost modeli tüm train verisi üzerinde eğitiliyor...
0:	learn: 0.7430215	total: 13.6ms	remaining: 

**HPO Kodu**

In [ ]:
import pandas as pd
import numpy as np
import optuna
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import gc

print("Adım 2b: Hiperparametre Optimizasyonu (Optuna ile) Başladı.")

# --- Veritabanı ve Çalışma Ayarları ---
DB_FILENAME = "optuna_studies.db"
STUDY_NAME = "catboost_v7_features" # Her yeni özellik seti için bu ismi değiştirebilirsin

train_path = "/content/datathon/processed/train_processed_v7.csv"

feature_path = "/content/models/V7/feature_importance/OUT/importance.json"

# --- Modelleme için Veriyi Hazırlama ---
# --- İşlenmiş Veriyi Yükleme ---
try:
    df_train = pd.read_csv(train_path, index_col='user_session')
    print("İşlenmiş train verisi ('train_processed_v7.csv') yüklendi.")

    with open(feature_path, 'r') as f:
      features_importance = json.load(f)
    print(f"Özelikler Başarıyla yüklendi... {len(features_importance)}")
except FileNotFoundError:
    print("Hata: 'train_processed_v7.csv' bulunamadı. Lütfen önce '1_data_preparation_v7.py' scriptini çalıştırın.")
    exit()

# --- 1. Adım: Özellik Seçilimi ---
print("\nÖzelikler Seçiliyor...")
importance_threshold = 0.2
selected_features = [item['feature'] for item in features_importance if item['importance'] >= importance_threshold]
X = df_train[selected_features]
print(f"Önemi >= {importance_threshold} olan {len(selected_features)} adet özellik seçildi.")

# --- Veriyi Hazırlama ---
y = df_train['session_value']
y_log = np.log1p(y)

# Aynı doğrulama setini kullanmak için ayırma işlemini tekrar yapıyoruz
X_train, X_val, y_train_log, y_val_log = train_test_split(
    X, y_log, test_size=0.2, shuffle=False
)

# Objective fonksiyonu (GÜNCELLENDİ)
def objective(trial):
    params = {
        'objective': 'RMSE',
        'iterations': 4500,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_seed': 42,
        'verbose': 0,
    }

    model = CatBoostRegressor(**params)
    # Modeli eğitirken hem train (X_train) hem de val (X_val) setini izlemesini sağlıyoruz
    model.fit(
        X_train, y_train_log,
        eval_set=[(X_train, y_train_log), (X_val, y_val_log)], # İki seti de ekledik
        early_stopping_rounds=250,
        verbose=0
    )

    # En iyi iterasyondaki skorları al
    scores = model.get_best_score()

    best_learn_rmse = scores['learn']['RMSE']
    val_rmse_0 = scores['validation_0']['RMSE']
    val_rmse_1 = scores['validation_1']['RMSE']

    # MSE'yi hesapla
    preds_log = model.predict(X_val)
    preds = np.expm1(preds_log)
    preds[preds < 0] = 0
    y_val_original = np.expm1(y_val_log)
    mse = mean_squared_error(y_val_original, preds)

    # --- GÜNCELLENMİŞ RAPORLAMA KISMI ---
    print(f"✅ Trial {trial.number} bitti | MSE: {mse:.4f} | Learn RMSE: {best_learn_rmse:.4f} | Val_0 RMSE: {val_rmse_0:.4f} | Val_1 RMSE : {val_rmse_1:.4f} | İterasyon: {model.get_best_iteration()}")
    gc.collect()
    return mse


# --- Optimizasyon Sürecini Başlatma (VERİTABANI İLE) ---
# SQLite veritabanı için bağlantı dizesi oluştur
storage_name = f"sqlite:///{DB_FILENAME}"

# Çalışmayı oluştur veya varsa veritabanından yükle
study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=storage_name,
    direction='minimize',
    load_if_exists=True # Eğer bu isimde bir çalışma varsa, sıfırdan başlamak yerine devam et
)

# Optimizasyonu çalıştır
n_trials = 100
print(f"Optimizasyon başlıyor... Sonuçlar '{DB_FILENAME}' dosyasına kaydedilecek.")
print(f"Mevcut deneme sayısı: {len(study.trials)}. Toplamda {n_trials} denemeye ulaşılacak.")
study.optimize(objective, n_trials=n_trials)

# --- Sonuçları Yazdırma ---
print("\nOptimizasyon Tamamlandı!")
print(f"Toplam deneme sayısı: {len(study.trials)}")
print(f"En iyi denemenin skoru (MSE): {study.best_value}")
print("En iyi denemenin parametreleri:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

Adım 2b: Hiperparametre Optimizasyonu (Optuna ile) Başladı.


[I 2025-08-27 10:47:59,429] A new study created in RDB with name: catboost_v7_features


İşlenmiş train verisi ('train_processed_v7.csv') yüklendi.
Özelikler Başarıyla yüklendi... 26

Özelikler Seçiliyor...
Önemi >= 0.2 olan 26 adet özellik seçildi.
Optimizasyon başlıyor... Sonuçlar 'optuna_studies.db' dosyasına kaydedilecek.
Mevcut deneme sayısı: 0. Toplamda 100 denemeye ulaşılacak.


[I 2025-08-27 10:48:32,322] Trial 0 finished with value: 279.0411378780158 and parameters: {'learning_rate': 0.0449482780779809, 'depth': 4, 'l2_leaf_reg': 1.629952328412843, 'colsample_bylevel': 0.6799031378424281, 'min_child_samples': 93}. Best is trial 0 with value: 279.0411378780158.


✅ Trial 0 bitti | MSE: 279.0411 | Learn RMSE: 0.4310 | Val_0 RMSE: 0.4310 | Val_1 RMSE : 0.4329 | İterasyon: 2198


...

✅ Trial 96 bitti | MSE: 274.4783 | Learn RMSE: 0.4321 | Val_0 RMSE: 0.4321 | Val_1 RMSE : 0.4329 | İterasyon: 2755
✅ Trial 97 bitti | MSE: 276.2445 | Learn RMSE: 0.4314 | Val_0 RMSE: 0.4314 | Val_1 RMSE : 0.4329 | İterasyon: 2968


[I 2025-08-27 11:33:46,111] Trial 97 finished with value: 276.2444732985654 and parameters: {'learning_rate': 0.033130834521821244, 'depth': 4, 'l2_leaf_reg': 1.816165412679661, 'colsample_bylevel': 0.7135149938737803, 'min_child_samples': 25}. Best is trial 37 with value: 252.98070837051267.
[I 2025-08-27 11:34:05,596] Trial 98 finished with value: 396.40675104939953 and parameters: {'learning_rate': 0.06149233454334478, 'depth': 10, 'l2_leaf_reg': 1.6420346788268794, 'colsample_bylevel': 0.5059557101086566, 'min_child_samples': 50}. Best is trial 37 with value: 252.98070837051267.


✅ Trial 98 bitti | MSE: 396.4068 | Learn RMSE: 0.4184 | Val_0 RMSE: 0.4184 | Val_1 RMSE : 0.4349 | İterasyon: 210


[I 2025-08-27 11:34:43,960] Trial 99 finished with value: 284.0597497572077 and parameters: {'learning_rate': 0.02394551588746408, 'depth': 5, 'l2_leaf_reg': 2.3200461910518952, 'colsample_bylevel': 0.823485176185526, 'min_child_samples': 45}. Best is trial 37 with value: 252.98070837051267.


✅ Trial 99 bitti | MSE: 284.0597 | Learn RMSE: 0.4307 | Val_0 RMSE: 0.4307 | Val_1 RMSE : 0.4331 | İterasyon: 2504

Optimizasyon Tamamlandı!
Toplam deneme sayısı: 100
En iyi denemenin skoru (MSE): 252.98070837051267
En iyi denemenin parametreleri:
    learning_rate: 0.06007502637954865
    depth: 4
    l2_leaf_reg: 2.5440079813104126
    colsample_bylevel: 0.8424763199702767
    min_child_samples: 29


**Model Eğitimi - Özellik Seçilimli**

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

print("Adım 2: Model Eğitimi Başlandı.")

MODEL_DIR = "/content/models/V7"
os.makedirs(MODEL_DIR, exist_ok=True)

IN_TRAIN_PATH = "/content/datathon/processed/train_processed_v7.csv"

FEATURE_IMPORTANCE_DIR_IN = MODEL_DIR + "/feature_importance/IN/"
os.makedirs(FEATURE_IMPORTANCE_DIR_IN, exist_ok=True)

IN_FEATURES_PATH = FEATURE_IMPORTANCE_DIR_IN + "importance.json"


FEATURE_IMPORTANCE_DIR_OUT = MODEL_DIR + "/feature_importance/OUT/"
os.makedirs(FEATURE_IMPORTANCE_DIR_OUT, exist_ok=True)

OUT_FEATURES_PATH = FEATURE_IMPORTANCE_DIR_OUT + "importance.json"
# --- İşlenmiş Veriyi Yükleme ---
try:
    df_train = pd.read_csv(IN_TRAIN_PATH, index_col='user_session')
    print("İşlenmiş train verisi ('train_processed.csv') yüklendi.")

    with open(IN_FEATURES_PATH, 'r') as f:
      features_in = json.load(f)

    print("Özellikler yüklendi.")
except FileNotFoundError:
    print("Hata: 'train_processed.csv' bulunamadı. Lütfen önce '1_data_preparation.py' scriptini çalıştırın.")
    exit()

# --- Modelleme için Veriyi Hazırlama ---
y = df_train['session_value']
X = df_train.drop(['session_value'], axis=1)


# Hedef değişkene log dönüşümü
y_log = np.log1p(y)

# feature seçilimi
importance_threshold = 0.2
print(f"Toplam özellik sayısı: {len(features_in)}")
features_in = [item['feature'] for item in features_in if item['importance'] >= importance_threshold]
print(f"Seçilen özellik sayısı: {len(features_in)}")
X = X[features_in]

# --- Zaman Bazlı Doğrulama (Time-Based Validation) ---
X_train, X_val, y_train_log, y_val_log = train_test_split(
    X, y_log, test_size=0.2, shuffle=False # shuffle=False zaman serisi doğrulama için kritik!
)
print(f"Train seti boyutu: {X_train.shape[0]}, Validation seti boyutu: {X_val.shape[0]}")

# --- CatBoost Modelini Eğitme ve Değerlendirme ---
print("CatBoost Modeli eğitimi başlıyor...")

best_params = {
    'learning_rate': 0.06007502637954865,
    'depth': 4,
    'l2_leaf_reg': 2.5440079813104126,
    'colsample_bylevel': 0.8424763199702767,
    'min_child_samples': 29,
    'objective': 'RMSE',
    'random_seed': 42,
    'verbose': 500
}

cat_model = CatBoostRegressor(
    **best_params,
    iterations=4500,
    eval_metric='RMSE',
    early_stopping_rounds=300
)


cat_model.fit(
    X_train, y_train_log,
    eval_set=(X_val, y_val_log)
)

# --- Performans Değerlendirme ---
val_preds_log = cat_model.predict(X_val)
val_preds = np.expm1(val_preds_log)
val_preds[val_preds < 0] = 0
y_val = np.expm1(y_val_log)

validation_mse = mean_squared_error(y_val, val_preds)
print(f"\nValidation Seti Üzerindeki MSE Skoru (CatBoost): {validation_mse:.4f}")
print(f"Validation Seti Üzerindeki RMSE Skoru (CatBoost): {np.sqrt(validation_mse):.4f}")

# --- Final Modelini Eğitme ve Kaydetme ---
print("\nFinal CatBoost modeli tüm train verisi üzerinde eğitiliyor...")
final_model = CatBoostRegressor(
    **best_params,
    iterations=cat_model.get_best_iteration()
)
final_model.fit(X, y_log)

features_out = X.columns.tolist()

# Feature importance as a DataFrame and save in a format that works
fi = final_model.get_feature_importance(prettified=False)
df_fi = pd.DataFrame({"feature": features_out, "importance": fi})
df_fi = df_fi.sort_values("importance", ascending=False).reset_index(drop=True)

# Prefer parquet, fallback to CSV if parquet not available, always also save JSON

df_fi.to_json(OUT_FEATURES_PATH, orient="records")



out_dir = "/content/models/V7/"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "catboost_model_v7_selected_1.cbm")

# Modeli kaydetme
final_model.save_model(out_path)

print("\nAdım 2 Tamamlandı: 'catboost_model_v7_selected.cbm' dosyası olarak model kaydedildi.")

Adım 2: Model Eğitimi Başlandı.
İşlenmiş train verisi ('train_processed.csv') yüklendi.
Özellikler yüklendi.
Toplam özellik sayısı: 28
Seçilen özellik sayısı: 26
Train seti boyutu: 56605, Validation seti boyutu: 14152
CatBoost Modeli eğitimi başlıyor...
0:	learn: 0.7429101	test: 0.7429800	best: 0.7429800 (0)	total: 10.4ms	remaining: 46.6s
500:	learn: 0.4364872	test: 0.4338891	best: 0.4338891 (500)	total: 4.74s	remaining: 37.8s
1000:	learn: 0.4336950	test: 0.4329969	best: 0.4329929 (997)	total: 9.42s	remaining: 32.9s
1500:	learn: 0.4320644	test: 0.4328699	best: 0.4328656 (1444)	total: 16.5s	remaining: 33s
2000:	learn: 0.4308195	test: 0.4329015	best: 0.4328412 (1762)	total: 21.1s	remaining: 26.4s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.4328411697
bestIteration = 1762

Shrink model to first 1763 iterations.

Validation Seti Üzerindeki MSE Skoru (CatBoost): 266.6631
Validation Seti Üzerindeki RMSE Skoru (CatBoost): 16.3298

Final CatBoost modeli tüm train veris

**Submission Oluşturma**

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from catboost import CatBoostRegressor

print("Adım 3 (Akıllı Doldurma ile): Tahmin ve Gönderim Başladı.")

# --- Ayarlar ---
PROCESSED_TEST_FILE = '/content/datathon/processed/test_processed_v7.csv'
MODEL_FILE = '/content/models/V7/catboost_model_v7_selected.cbm' # v3 için optimize edilmiş modeli kullandığımızdan emin olalım
SELECTED_FEATURES_PATH = '/content/models/V7/feature_importance/OUT/importance.json'
TRAIN_RAW_PATH = '/content/datathon/train.csv' # Sızıntı tespiti için gerekli
TEST_RAW_PATH = '/content/datathon/test.csv'   # Sızıntı ve anomali tespiti için gerekli

SUBMISSION_DIR = "/content/submissions"
os.makedirs(SUBMISSION_DIR, exist_ok=True)
SUBMISSION_FILE = '/content/submissions/submission_v7.csv'


# --- Gerekli Dosyaları Yükleme ---
try:
    df_test_processed = pd.read_csv(PROCESSED_TEST_FILE) # index_col olmadan okuyoruz
    df_submission = pd.read_csv('/content/datathon/sample_submission.csv')
    df_train_raw = pd.read_csv(TRAIN_RAW_PATH)
    df_test_raw = pd.read_csv(TEST_RAW_PATH)

    with open(SELECTED_FEATURES_PATH, 'r') as f:
      selected_features_dict = json.load(f)

    model = CatBoostRegressor()
    model.load_model(MODEL_FILE)

    print(f"Gerekli dosyalar ve {len(selected_features_dict)} adet seçilmiş özellik başarıyla yüklendi.")
except FileNotFoundError as e:
    print(f"Hata: {e}.")
    exit()

# --- 1. ADIM: DOĞRULANMIŞ VERİ SIZINTISINI TESPİT ET ---
print("\nDoğrulanmış veri sızıntısı tespit ediliyor...")
train_session_users = df_train_raw.groupby('user_session')['user_id'].apply(set)
test_session_users = df_test_raw.groupby('user_session')['user_id'].apply(set)
common_sessions = set(train_session_users.index).intersection(set(test_session_users.index))
verified_leaked_sessions = {sid for sid in common_sessions if train_session_users[sid] == test_session_users[sid]}
verified_leak_map = df_train_raw[
    df_train_raw['user_session'].isin(verified_leaked_sessions)
].groupby('user_session')['session_value'].first().to_dict()
print(f"Tespit edilen DOĞRULANMIŞ sızıntı seans sayısı: {len(verified_leaked_sessions)}")


# --- 2. ADIM: MODEL TAHMİNLERİNİ YAP --
selected_features = [item['feature'] for item in selected_features_dict]

# Olası eksik sütun hatalarını önlemek için kontrol
missing_cols = set(selected_features) - set(df_test_processed.columns)
if missing_cols:
    print(f"HATA: Test verisinde şu sütunlar eksik: {missing_cols}")
    exit()

# Sütun sırasının modelin beklediğiyle aynı olmasını garantile
print("Selected Feature Sayısı : ", len(selected_features))
# Test verisini filtrele
df_test_processed.set_index('user_session', inplace=True)
df_test_selected = df_test_processed[selected_features]

# Tahmin yap
print("Test seti üzerinde model tahminleri yapılıyor...")
test_preds_log = model.predict(df_test_selected)
final_predictions = np.expm1(test_preds_log)
final_predictions[final_predictions < 0] = 0
df_predictions = pd.DataFrame({
    'user_session': df_test_selected.index,
    'predicted_value': final_predictions
})

# --- 3. ADIM: SUBMISSION DOSYASINI OLUŞTURMA ---
# Önce normal model tahminlerini map et
submission_map = dict(zip(df_predictions['user_session'], df_predictions['predicted_value']))
df_submission['session_value'] = df_submission['user_session'].map(submission_map)

# Sonra "Akıllı Doldurma" ile NaN user anormalliklerini doldur
session_user_counts = df_test_raw.groupby('user_session')['user_id'].nunique()
anomalous_sessions_orig = session_user_counts[session_user_counts > 1].index.tolist()
print(f"\nAkıllı doldurma yapılacak anormal seanslar: {anomalous_sessions_orig}")
for session_id in anomalous_sessions_orig:
    constituent_preds = df_predictions[df_predictions['user_session'].str.startswith(f"{session_id}_")]
    total_value = constituent_preds['predicted_value'].sum()
    df_submission.loc[df_submission['user_session'] == session_id, 'session_value'] = total_value
    print(f"'{session_id}' için {len(constituent_preds)} parçanın tahmini toplandı: {total_value:.4f}")

# SON VE EN ÖNEMLİ ADIM: Doğrulanmış sızıntıdan gelen gerçek değerlerle tüm tahminleri EZ
print(f"\n{len(verified_leaked_sessions)} adet seansın değeri, sızıntıdan gelen gerçek değerlerle güncelleniyor...")
leaked_updates = df_submission['user_session'].map(verified_leak_map)
df_submission['session_value'] = np.where(leaked_updates.notna(), leaked_updates, df_submission['session_value'])

# --- SON KONTROLLER ---
nan_count = df_submission['session_value'].isnull().sum()
if nan_count > 0:
    print(f"⚠️ UYARI: Hala {nan_count} adet null değer var! Bunlar 0 ile dolduruluyor.")
    df_submission['session_value'].fillna(0, inplace=True)

excepted_row = 30789
if excepted_row != len(df_submission):
    print(f"Satır sayıları eşleşmiyor. Beklenen : {excepted_row}. Bulunan : {len(df_submission)}")
    exit()

# --- Dosyayı Kaydetme ---
df_submission.to_csv(SUBMISSION_FILE, index=False)
print(f"\n'{SUBMISSION_FILE}' dosyası başarıyla oluşturuldu! Bu son gönderim için bol şans!")
print("Dosyanın ilk 5 satırı:")
print(df_submission.head())

Adım 3 (Akıllı Doldurma ile): Tahmin ve Gönderim Başladı.
Gerekli dosyalar ve 26 adet seçilmiş özellik başarıyla yüklendi.

Doğrulanmış veri sızıntısı tespit ediliyor...
Tespit edilen DOĞRULANMIŞ sızıntı seans sayısı: 209
Selected Feature Sayısı :  26
Test seti üzerinde model tahminleri yapılıyor...

Akıllı doldurma yapılacak anormal seanslar: ['SESSION_000000', 'SESSION_094821']
'SESSION_000000' için 17 parçanın tahmini toplandı: 528.7005
'SESSION_094821' için 2 parçanın tahmini toplandı: 126.5101

209 adet seansın değeri, sızıntıdan gelen gerçek değerlerle güncelleniyor...

'/content/submissions/submission_v7.csv' dosyası başarıyla oluşturuldu! Bu son gönderim için bol şans!
Dosyanın ilk 5 satırı:
     user_session  session_value
0  SESSION_164059     181.316098
1  SESSION_109583      44.978012
2  SESSION_171382      39.932005
3  SESSION_137110      28.218247
4  SESSION_146503     164.923443
